In [1]:
import open3d as o3d
import numpy as np
import cv2

def extract_image_and_points(cloud, cloud_height, cloud_width, flip_rgb=False):
    points = np.asarray(cloud.points)
    color = np.asarray(cloud.colors) * 255
    img = color.reshape((cloud_height, cloud_width, 3))
    if flip_rgb:
        img = img[:, :, ::-1].copy()
    points = points.reshape((cloud_height, cloud_width, 3))
    return img, points

In [2]:
import os 
def traverse_files(rootDir): 
    files_list = []
    list_dirs = os.walk(rootDir) 

    for root, dirs, files in list_dirs:    
        for f in files:
            files_list.append(os.path.join(root, f))
    return files_list

In [3]:
pcd_paths = traverse_files('/home/bot/dev/app/cbee_app/tote_full_facade/example/setup1/denoise/1A_14cm')

for path in pcd_paths:
    pcd = o3d.io.read_point_cloud(path, remove_nan_points=False)
    if np.array(pcd.colors).size != 480*640*3:
        continue

    image, points = extract_image_and_points(pcd, 480, 640, True)
    
    (h, w) = image.shape[:2]
    center = (w//2, h//2)
    M = cv2.getRotationMatrix2D(center, 180, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))

    cv2.imwrite(path[:-3]+'png', rotated)
#     break